# Image Recognition

Verrà utilizzato YOLO nella versione 11 con modello nano. 

## ✅ Step 1 – Preparazione
Assicurarsi che la libreria `ultralytics` sia installata

In [ ]:
%pip install --quiet ultralytics

``base_dir`` rappresenta il percorso assoluto del dataset mentre ``workers`` rappresenta il numero di thread che il progetto potrà usare quando incontra funzioni multithread. Attenzione che fa un sacco di overhead se si imposta il parametro ``workers`` di YOLO ad un valore elevato.

In [1]:
import os 

base_dir = "/mnt/e/objects365"
workers = max(4, os.cpu_count() - 2) # 22 sul mio PC

Il dataset scaricato non ha la cartella di test ma solo `train` e `val` che sono per appunto addestramento e validazione. Ho deciso di creare una cartella di test togliendo dalla cartella di validazione alcune patch. 

In [ ]:
from data.create_test_samples import create_test_set

create_test_set(base_dir, test_size_ratio=0.1, num_threads=8)

Le patch sono molto grandi e per limiti hardware ho deciso di fare uno split, quindi sia labels che images che test avranno il doppio delle patch ma grandi la metà dove ogni patchX sarà divisa in patchX_a e patchX_b doce la X è un numero intero.

In [ ]:
from data.split_patch import dividi_patch_in_due_multithread
import os

workers = max(4, os.cpu_count() - 2)

# Gestisco le Labels
dividi_patch_in_due_multithread(f"{base_dir}/labels/test", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/labels/val", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/labels/train", max_workers=workers)

# Gestisco le immagini
dividi_patch_in_due_multithread(f"{base_dir}/images/test", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/images/train", max_workers=workers)
dividi_patch_in_due_multithread(f"{base_dir}/images/val", max_workers=workers)


Siccome YOLO ha bisogno di un file di configurazione in formato YAML (Yet Another Markup Language) e siccome il dataset è diviso in patch c'è bisogno di dare le coordinate delle patch a YOLO e per farlo bisogna creare un file TXT con tutti i percorsi assoluti di tutti i file da processare.

In [ ]:
from data.create_image_lists import create_patch_image_lists


# Crea gli elenchi delle immagini per il training, la validazione e il test
threads = 16
create_patch_image_lists(base_dir, "train", f"{base_dir}/patch_txts", num_threads=threads)
create_patch_image_lists(base_dir, "val", f"{base_dir}/patch_txts", num_threads=threads)
create_patch_image_lists(base_dir, "test", f"{base_dir}/patch_txts", num_threads=threads)

Qui un estratto del file YAML:

```yaml
path: /mnt/e/objects365/patch_txts # Percorso base
train: train_patch0_a.txt  # File con elenco immagini di training
val: val_patch0_a.txt      # File con elenco immagini di validation
```
Per automatizzare il processo ho creato uno script per gestire dinamicamente il file YAML. 
Attenzione! Il numero di patch di validazione è inferiore a quello di training quindi alcuni passaggi di train saranno validati con la stessa patch di validazione. 

Mi sono accorto che nelle etichette spesso compare la classe 365 ma questa non è mappata nel file YAML e siccome YOLO non ha la classe `background` come default per le classi sconosciute allora per identificare tutto ciò che non è mappabile dal modello, ho deciso di mappare la classe 365 delle etichette come classe `Unknown`.

```yaml
# Classes
names:
  0: Person
  1: Sneakers
  2: Chair
  3: Other Shoes
  ...
  361: Lipstick
  362: Cosmetics Mirror
  363: Curling
  364: Table Tennis
  365: Unknown
```

## ✅ Step 2 – Addestramento del Modello

Il modello di YOLO scelto sarà un *pretrained*, successivamente verrà importato il modello addestrato sulle patch del dataset. Ho scelto il modello nano perché compatibile insieme alla dimensine delle patch per girare sulla mia GPU, anche solo con il modello small la quantità di VRAM necessaria eccede quella della mia GPU e andando ad utilizzare della memoria condivisa in RAM il processo rallenta drasticamente al punto di passare da iterazioni al secondo a secondi per iterazione.


In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

## Augmentation Parameters
Gli aumenti dei dati sono tecniche utilizzate per migliorare la robustezza e la generalizzazione del modello. Ecco i parametri configurabili:

- **hsv_h (Hue)**: Regola la tonalità dei colori per rendere il modello invariante alle variazioni di colore.
- **hsv_s (Saturation)**: Modifica la saturazione per simulare diverse condizioni di illuminazione.
- **hsv_v (Brightness)**: Cambia la luminosità per gestire immagini con diverse esposizioni.
- **degrees**: Applica rotazioni casuali per migliorare l'invarianza al punto di vista.
- **translate**: Introduce traslazioni casuali per rendere il modello robusto a oggetti in posizioni diverse.
- **scale**: Ridimensiona gli oggetti per gestire variazioni di dimensioni.
- **shear**: Applica tagli (shearing) per simulare distorsioni geometriche (impostato a 0 per evitare distorsioni estreme).
- **fliplr**: Effettua un flip orizzontale casuale per aumentare la varietà dei dati.
- **mosaic**: Combina più immagini in un'unica immagine per migliorare il rilevamento di piccoli oggetti e il contesto.
- **bgr**: Scambia i canali BGR per gestire diversi formati di immagine.

In [16]:
import os
data_yaml = "data/objects365_step.yaml"
train_params = {
    "data":         data_yaml,               # YAML con path /mnt/e/yolo-fast
    "epochs":       20,                     # numero di epoche
    "patience":     5,                      # early stopping
    "batch":        16,                      # auto batch size (~60% VRAM)
    "imgsz":        640,                     # dimensione delle immagini
    "device":       0,                       # GPU 0
    "workers":      4,                       # DataLoader parallelismo
    "cache":        False,                   # caching su disco (/mnt/e)
    "save":         True,                    # salva checkpoint e modello
    "save_period":  5,                       # checkpoint ogni 5 epoche
    "project":      "runs/train",            # cartella di uscita
    "name":         "exp_fastcache",         # nome del run
    "exist_ok":     True,                    # sovrascrivi se esiste
    "pretrained":   True,                    # usa pesi pretrained
    "optimizer":    "AdamW",                 # SGD o Adam/AdamW
    "amp":          True,                    # mixed precision
    "rect":         False,                   # rectangular training off
    "multi_scale":  True,                    # multi-scale training on
    "cos_lr":       True,                    # cosine LR scheduler
    "close_mosaic": 10,                      # disabilita mosaic ultime 10 epoche
    "lr0":          0.0001,                    # learning rate iniziale
    "lrf":          0.1,                     # final LR = lr0 * lrf
    "momentum":     0.937,                   # momentum / beta1 per Adam
    "weight_decay": 0.0005,                  # regularizzazione L2
    "warmup_epochs":    3.0,                 # epoche warmup LR
    "warmup_momentum":  0.8,                 # momentum warmup
    "warmup_bias_lr":   0.1,                 # bias LR warmup
    "seed":         42,                      # per riproducibilità
    "fraction":     1.0,                     # usa 100% del dataset
    "val":          True,                    # abilita validazione
    "plots":        False                    # salva grafici di training/val
}

In [17]:
# Parametri di data augmentation ottimizzati
augmentation_params = {
    # Colore (HSV)
    "hsv_h":       0.015,   # Default: 0.015 (range 0.0–1.0) – mantiene piccole variazioni di tonalità
    "hsv_s":       0.7,     # Default: 0.7   (range 0.0–1.0) – variazione consistente di saturazione
    "hsv_v":       0.4,     # Default: 0.4   (range 0.0–1.0) – variazione di luminosità medio-alta

    # Geometriche
    "degrees":     5.0,     # Da 0.0 a 180 – rotazioni lievi per preservare orientamenti realistici
    "translate":   0.1,     # Da 0.0 a 1.0   – spostamenti fino al 10% delle dimensioni d’immagine
    "scale":       0.3,     # ≥ 0.0          – riduzione del range (da 0.5 a 0.3) per minor carico computazionale
    "shear":       0.0,     # –180 a +180    – mantenuto a 0 per evitare distorsioni drastiche
    "perspective": 0.0005,  # 0.0–0.001      – inserita una lieve prospettiva per aumentare la robustezza 3D

    # Flip
    "flipud":      0.0,     # 0.0–1.0 – no flip verticale (rischio di immagini “capovolte” poco realistiche)
    "fliplr":      0.5,     # 0.0–1.0 – flip orizzontale con probabilità 50%

    # Canale BGR
    "bgr":         1.0,     # 0.0–1.0 – scambio completo dei canali per insegnare a gestire l’ordine BGR

    # Mosaic / MixUp / CutMix
    "mosaic":      0.5,     # 0.0–1.0 – mosaic ridotto al 50% per bilanciare velocità e varietà
    "mixup":       0.3,     # 0.0–1.0 – mixup moderato per label noise e greater generalization

    # Classification-only (opzionali, ignora se non serve)
    # "erasing":   0.4,     # 0.0–0.9 – random erasing (classification only, lasciare a default se non serve)
    # "auto_augment": "randaugment",  # classification only
}
train_params.update(augmentation_params)


Gli addestramenti vengono fatti su ogni patch e per automatizzare il processo tengo traccia su un file JSON sia il numero di patch che la sua lettera. 

In [15]:
import importlib
import data.tracking_training
importlib.reload(data.tracking_training)
from data.tracking_training import leggi_e_incrementa_numero_json, aggiorna_train_val_yaml
traking_number, letter_patch = leggi_e_incrementa_numero_json(filepath="data/tracking_number_and_patch_letter.json")

val_number = traking_number - 1 if traking_number in [1, 6, 15, 24] else traking_number # alcuni val mancano perché ora sono test
print(f"Tracking number: {traking_number}, Letter patch: {letter_patch}, Validation number: {val_number}")
aggiorna_train_val_yaml(
    yaml_path="data/objects365_step.yaml",
    nuovo_train=f"/mnt/e/objects365/patch_txts/train_patch{traking_number}_{letter_patch}.txt",
    nuovo_val=f"/mnt/e/objects365/patch_txts/val_patch{val_number}_{letter_patch}.txt"
)

Tracking number: 4, Letter patch: a, Validation number: 4


Ecco l'addestramento del modello:
1. Carico il modello
    - Se primo giro allora la versione base di YOLO nano
    - Altrimenti il modello addestrato precedentemente
2. Lancio l'addestramento con i parametri impostati precedentemente
3. Salvo il modello addestrato
4. Salvo le metriche di valutazione su file.

In [18]:
import json
from ultralytics import YOLO

prev_letter_patch = 'a' if letter_patch == 'b' else 'b'
prev_traking_number = traking_number - 1 if letter_patch == 'a' else traking_number

if traking_number == 0 and letter_patch == 'a':
    train_params["resume"] = False  # Non riprendo da un modello precedente
    model = YOLO("yolo11n.pt")  # Load a pretrained model (recommended for training)
    print("Starting training from scratch with YOLOv11n model.")
else:
    train_params["resume"] = False
    model = YOLO(f"results/best_model_{prev_traking_number}_{prev_letter_patch}.pt")  # carico addestramento precedente
    print(f"Resuming training from previous model: results/best_model_{prev_traking_number}_{prev_letter_patch}.pt")
    
# Addestra il modello con gli aumenti dei dati
results = model.train(**train_params)
# Crea la cartella 'results' se non esiste
# Percorso per salvare i file
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

model.save(f"{output_dir}/best_model_{traking_number}_{letter_patch}.pt")  # Salva il modello addestrato

# Nome base dei file
filename = f"{output_dir}/training_results_{traking_number}_{letter_patch}"

# Salvataggio in TXT
metrics = results.results_dict if hasattr(results, "results_dict") else {}
with open(f"{filename}.txt", "w") as f:
    f.write(f"Training {traking_number}_{letter_patch} Results:\n")
    f.write(f"mAP50: {metrics.get('mAP50', 0.0):.4f}\n")
    f.write(f"mAP50-95: {metrics.get('mAP50-95', 0.0):.4f}\n")
    f.write(f"Precision: {metrics.get('precision', 0.0):.4f}\n")
    f.write(f"Recall: {metrics.get('recall', 0.0):.4f}\n")
    f.write(f"Training Time: {metrics.get('training_time', 'N/A')}\n")

# Salvataggio in JSON SOLO delle metriche
with open(f"{filename}.json", "w") as f_json:
    json.dump(metrics, f_json, indent=4)

print(f"✅ Risultati di addestramento salvati in '{filename}.txt' e '{filename}.json'")


Resuming training from previous model: results/best_model_3_b.pt
Ultralytics 8.3.146 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=1.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/objects365_step.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.3, mode=train, model=results/best_model_3_b.pt, momentum=0.937, mosaic=0.5, multi_scale=True, name=exp_fastcache, nbs=64, nms=False, opset=None, optimize=False, optim

train: Scanning /mnt/e/objects365/labels/train/patch4_a... 17260 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17260/17260 [00:53<00:00, 325.56it/s]

train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00184877.jpg: 2 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00184919.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00184937.jpg: 5 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185029.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185047.jpg: 4 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185093.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185151.jpg: 5 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185156.jpg: 1 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185282.jpg: 2 duplicate labels removed
train: /mnt/e/objects365/images/train/patch4_a/objects365_v1_00185316.jpg: 1 dupli

train: New cache created: /mnt/e/objects365/labels/train/patch4_a.cache
val: Fast image access ✅ (ping: 2.3±0.5 ms, read: 11.5±2.2 MB/s, size: 104.6 KB)


val: Scanning /mnt/e/objects365/labels/val/patch4_a... 798 images, 0 backgrounds, 0 corrupt: 100%|██████████| 798/798 [00:02<00:00, 352.83it/s]

...

                   all        798      12918      0.445      0.119      0.113     0.0653
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      11/20      11.2G      1.424      1.814       1.29        227        768: 100%|██████████| 1079/1079 [02:09<00:00,  8.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:04<00:00,  5.94it/s]


                   all        798      12918      0.453      0.122      0.112      0.063
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 6, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

11 epochs completed in 0.429 hours.
Optimizer stripped from runs/train/exp_fastcache/weights/last.pt, 5.9MB
Optimizer stripped from runs/train/exp_fastcache/weights/best.pt, 5.9MB

Validating runs/train/exp_fastcache/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO11n summary (fused): 100 layers, 2,728,186 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:03<00:00,  7.50it/s]


                   all        798      12918      0.464      0.118      0.116      0.066
              Sneakers        315        817       0.24      0.563      0.334      0.182
                 Chair         69        155       0.23      0.406      0.274      0.136
           Other Shoes        160        266      0.173      0.545      0.336      0.201
                   Car         52         80      0.231       0.45      0.334      0.209
                  Lamp         33         58       0.13      0.328      0.105     0.0636
               Glasses        145        253      0.339      0.466      0.385      0.249
                Bottle         44         60      0.198      0.217      0.192     0.0864
                  Desk        162        315      0.181      0.254      0.127     0.0654
                   Cup        177        229      0.166       0.52      0.249      0.116
         Street Lights        120        194      0.284       0.36      0.232      0.137
         Cabinet/shel

✅ Risultati di addestramento salvati in 'results/training_results_1.txt' e 'results/training_results_1.json'


## ✅ Step 3 – Validazione del Modello

In [ ]:
print(model.info())

In [ ]:
import os
from ultralytics import YOLO
import json

# Path al file di pesi salvati
weights_path = "results/best_model_0.pt"

# ----------------------------
# 1. Determina se 'model' è già in memoria
# ----------------------------
try:
    # Prova ad accedere a `model`; se non esiste, scatena NameError
    _ = model
    in_ram = True
except NameError:
    in_ram = False

# ----------------------------
# 2. Scegli l'oggetto YOLO per la validazione
# ----------------------------
if in_ram:
    print("✔️ Modello già in RAM: lo uso per la validazione")
    model_to_val = model
elif os.path.exists(weights_path):
    print(f"✔️ Carico modello da disco: {weights_path}")
    model_to_val = YOLO(weights_path)
else:
    raise FileNotFoundError(f"Né modello in RAM né file trovato: {weights_path}")


# Esegui la validazione
val_results = model_to_val.val(
    data=data_yaml,
    imgsz=640,
    batch=16,
    device=0,
    save=True
)

# Crea la cartella dei risultati
os.makedirs("results", exist_ok=True)

# Ottieni il dizionario con le metriche
metrics = val_results.results_dict


filename = "results/validation_results_step_0"

# Stampa tutte le chiavi disponibili per debug (facoltativo)
print("Chiavi disponibili in results_dict:")
for k in metrics:
    print(f"- {k}")

# Salvataggio in TXT
with open(f"{filename}.txt", "w") as f:
    f.write("Validation Results:\n")
    f.write(f"mAP50: {metrics.get('mAP50', 0.0):.4f}\n")
    f.write(f"mAP50-95: {metrics.get('mAP50-95', 0.0):.4f}\n")
    f.write(f"Precision: {metrics.get('precision', 0.0):.4f}\n")
    f.write(f"Recall: {metrics.get('recall', 0.0):.4f}\n")
    f.write(f"Inference Speed: {val_results.speed.get('inference', 0.0):.2f} ms/img\n")

# Salvataggio in JSON
with open(f"{filename}.json", "w") as f_json:
    json.dump(metrics, f_json, indent=4)

print(f"✅ Risultati salvati in '{filename}.txt' e '{filename}.json'")
